In [1]:
import numpy as np
import gdspy
import os
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely
import tqdm
import matplotlib.image
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
chip = gdspy.GdsLibrary(infile='130nm_edit_0001.gds')

In [ ]:
gdspy.LayoutViewer(chip,pattern={'default': 8},background='#FFFFFF')

In [ ]:
gdspy.get_gds_units(infile='130nm_edit_0001.gds')

In [ ]:
keys=chip.cells.keys()
keys_l=list(keys)
# for key in keys:
#     path = os.path.join('layers_gdspy',key+'.svg')
#     print(path)
#     chip.cells[key].write_svg(path,scaling=0.1)
    

In [ ]:
top_io=chip.cells['top_io']

In [ ]:
poly_top = top_io.get_polygons()

In [ ]:
print(len(poly_top))

In [ ]:
top_layer = chip.top_level()

In [ ]:
def poly_intersection(A,B):
    #this function will return the intersection of the two polygons
    if A.intersects(B)==False:
        return False ,[]
    else:
        p=A.intersection(B)
        cord = [] # A list to contain the polygon arays 
        if p.geom_type == 'Polygon':
            xx, yy = p.exterior.coords.xy
            A =np.array([xx.tolist(),yy.tolist()]).transpose()
            cord.append(A)
        elif p.geom_type == 'MultiPolygon':
            for pi in p.geoms:
                xx, yy = pi.exterior.coords.xy
                A =np.array([xx.tolist(),yy.tolist()]).transpose()
                cord.append(A)
    return True, cord

In [ ]:
test_poly = Polygon([(493, 761), (493, 775), (700, 775), (700, 761)])

filtered_polygons = []
for f in poly_top:
    res, poly = poly_intersection(test_poly,Polygon(f))
    if res:
        filtered_polygons.append(f)
        

print(f"{len(filtered_polygons)} - found")

In [ ]:
# Step 1: Load the GDS file
gds_file = "130nm_edit_0001.gds"  # Replace with your GDS file path
gds_lib = gdspy.GdsLibrary(infile=gds_file)

# Step 2: Get the cell you want to examine
cell_name = "top_io"  # Replace with the name of the top-level cell
cell = gds_lib.cells[cell_name]

# Step 3: Define the bounding box for the section of interest
# Example bounding box (xmin, ymin, xmax, ymax)
section_bb = Polygon([(493, 682), (493, 857), (788, 857), (788, 682)])  # Replace with your section coordinates

# Extract polygons grouped by layer and datatype
polygons_by_layer = cell.get_polygons(by_spec=True)

# Filter polygons by bounding box
filtered_polygons = {}

for (layer, datatype), polygons in polygons_by_layer.items():
    for polygon in polygons:
        res, poly = poly_intersection(test_poly,Polygon(polygon))
        if res:
            if (layer, datatype) not in filtered_polygons:
                filtered_polygons[(layer, datatype)] = []
            for p in poly:
                filtered_polygons[(layer, datatype)].append(p)

# Print the number of polygons found in each layer
for (layer, datatype), polygons in filtered_polygons.items():
    print(f"Layer: {layer}, Datatype: {datatype}, Polygons: {len(polygons)}")

In [ ]:
# Create a new cell to hold the filtered polygons
# section_cell = gdspy.Cell('SECTION_VIEW')
gds_lib = gdspy.GdsLibrary()

# Add filtered polygons to the new cell
sections=[]
top_cell = gdspy.Cell('top_r')
for (layer, datatype), polygons in filtered_polygons.items():
    section_cell = gdspy.Cell(str((layer, datatype)))
    for polygon in polygons:
        section_cell.add(gdspy.Polygon(polygon, layer=layer, datatype=datatype))
        top_cell.add(gdspy.Polygon(polygon, layer=layer, datatype=datatype))
    sections.append(section_cell)
sections.append(top_cell)
# Visualize the section using the layout viewer
# gdspy.LayoutViewer()

# Optional: Save the section to a new GDS file
gds_lib.write_gds('section_of_chip.gds', cells=sections)


In [ ]:
chip_r = gdspy.GdsLibrary(infile='section_of_chip.gds')
keys=chip_r.cells.keys()
keys_l=list(keys)
for key in keys:
    path = os.path.join('layers_gdspy',key+'.svg')
    print(path)
    chip_r.cells[key].write_svg(path,scaling=100)

In [ ]:
chip_r = gdspy.GdsLibrary(infile='section_of_chip.gds')
gdspy.LayoutViewer()

# Starting from the reduced chip

In [2]:
def test_poly(p,l,t):
    targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
    poly=gdspy.Polygon(p-targ)
    if (poly.area()<0.01):
        return True
    else:
        return False
    
    

In [3]:
gds_lib = gdspy.GdsLibrary()
chip_r = gdspy.GdsLibrary(infile='section_of_chip.gds')
targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
cell = gdspy.Cell('target')
cell.add(gdspy.Polygon(targ, layer=500, datatype=500))
chip_r.cells['(65, 0)']
chip_r.cells['target']=cell
chip_r.cells['(65, 0)'].remove_polygons(test_poly)
gds_lib.write_gds('section_of_chip_t.gds', chip_r)

In [ ]:
list_cost_keys = list(chip_r.cells.keys())
list_cost_keys.remove("top_r")
list_cost_keys.remove("(127, 0)")
list_cost_keys.remove("target")
print(list_cost_keys)
list_reward_keys=list()
list_reward_keys.append("target")

In [ ]:
# This cell will build the basic mask
section_bb = Polygon([(493, 682), (493, 857), (788, 857), (788, 682)])  # Replace with your section coordinates
points = section_bb.exterior.xy
min_x, max_x = int(min(points[0])),int(max(points[0]))
min_y, max_y = int(min(points[1])),int(max(points[1]))
w = int(max_x - min_x)
h = int(max_y - min_y)
reselution = 100
mask_cost = np.zeros((w*reselution,h*reselution))
mask_reward = np.zeros((w*reselution,h*reselution))

In [ ]:
for t in list_cost_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_cost, pts=[ver], color=(255,0,0))

matplotlib.image.imsave('mask_cost.png', mask_cost)

for t in list_reward_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_reward, pts=[ver], color=(255,0,0))

matplotlib.image.imsave('mask_reward.png', mask_reward) 


In [ ]:
W,H = mask_cost.shape

In [ ]:
W

# Testing the Gaussian blurring

In [ ]:
blur_cost = cv2.GaussianBlur(mask_cost,(61,61),0)
blur_reward = cv2.GaussianBlur(mask_reward,(61,61),0)
img = blur_cost - blur_reward


In [ ]:
max_val = np.max(img)
min_val = np.min(img)
print(max_val,min_val)
img_norm = 255*(img +(-min_val))/(max_val-min_val)

In [ ]:
matplotlib.image.imsave('img_total.png', img_norm)


# building the GA alg

In [ ]:
def cost_func(X):
    x=int(X[0])
    y=int(X[1])
    w=int(X[2])
    h=int(X[3])
    cost=sum(sum(mask_cost[(x-w):(x+w),(y-h):(y+h)]))
    reward=sum(sum(mask_reward[(x-w):(x+w),(y-h):(y+h)]))
    if cost>0:
        return cost
    else:
        return -reward

In [ ]:
algorithm_param = {'max_num_iteration': 1000,\
                   'population_size':100,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

In [ ]:
varbound=np.array([[100,W-100],[100,H-100],[5,100],[5,100]])
model=ga(function=cost_func,dimension=4,variable_type='int',variable_boundaries=varbound,algorithm_parameters=algorithm_param)
model.run()

In [ ]:
x,y,w,h=model.best_variable/reselution


In [ ]:
x_g=x+493
y_g=y+682

In [ ]:
res_poly=[[x_g-w, y_g-h],
    [x_g-w, y_g+h],
    [x_g+w, y_g+h],
    [x_g+w, y_g-h],
    [x_g-h, y_g-h]]
cell = gdspy.Cell('res')
chip_r.cells['res']=cell
cell.add(gdspy.Polygon(res_poly, layer=555, datatype=555))
chip_r.cells['top_r'].add(gdspy.Polygon(res_poly, layer=555, datatype=555))
gds_lib.write_gds('section_of_chip_res.gds', chip_r)

In [ ]:
chip_r.cells['target'].add(gdspy.Polygon(res_poly, layer=555, datatype=555))
gds_lib.write_gds('section_of_chip_res.gds', chip_r)

In [ ]:
chip_r.cells.keys()

In [ ]:
print(x_g,y_g)

In [4]:
gdspy.(infile='section_of_chip.gds')

(1e-06, 1e-09)